In [ ]:
import lightgbm as lgb  # standard alias
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import math
from catboost import CatBoostRegressor
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

# Define variables for model
train = pd.read_csv('feature_data/training_set_dropped.csv')

x_test = pd.read_csv('feature_data/testing_set_dropped.csv')
x_train = train.drop('revenue', axis=1)
y_train = train['revenue']

x_train, x_test, y_train, y_test = train_test_split(train.drop('revenue', axis=1), train['revenue'], random_state=104, train_size=0.8, shuffle=True)


# Save store ids, then remove this column
store_ids = x_test['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True )
# Drop store id
x_test = x_test.drop('store_id', axis=1)
x_train = x_train.drop('store_id', axis=1)

#drop all categorical features until i know how  to do it
x_test = x_test.drop('chain_name', axis=1)
x_train = x_train.drop('chain_name', axis=1)
x_test = x_test.drop('mall_name', axis=1)
x_train = x_train.drop('mall_name', axis=1)
x_test = x_test.drop('lv3', axis=1)
x_train = x_train.drop('lv3', axis=1)
x_test = x_test.drop('lv2', axis=1)
x_train = x_train.drop('lv2', axis=1)
x_test = x_test.drop('lv1', axis=1)
x_train = x_train.drop('lv1', axis=1)




def save_submission(pred, name):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv(name, index=False)
    
def category(data):
    data['chain_name'] = data['chain_name'].astype('category')
    data['mall_name'] = data['mall_name'].astype('category')
    data['lv1'] = data['lv1'].astype('category')
    data['lv2'] = data['lv2'].astype('category')
    data['lv3'] = data['lv3'].astype('category')
#category(train)
#category(x_test)
#x_train['lv1'] = pd.Series(x_train['lv1'], dtype="category")
#x_test['lv1'] = pd.Series(x_test['lv1'], dtype="category")




In [ ]:
def catboost(x_train, y_train, x_test):
    model = CatBoostRegressor(iterations=1000,
                            learning_rate=0.01,
                            depth=12,
                            loss_function= 'RMSE')
    # Fit model
    model.fit(x_train, y_train)
    # Get predictions
    preds = model.predict(x_test)
    preds = np.expm1(preds)
    save_submission(preds, "cat.csv")

In [ ]:
def lgbm(x_train, x_test, y_train, y_test = None):
    #Hyper parameters
    hyper_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': ['l1','l2'],
        'learning_rate': 0.1,
        'bagging_freq': 10,
        'verbose': 0,
        "max_depth": 12,
        "num_leaves": 128,  
        "max_bin": 512,
        "num_iterations": 1000,
    }
    categorical = ['chain_name', 'mall_name', 'lv3']

    #fit the model takes some time
    gbm = lgb.LGBMRegressor(**hyper_params)
    gbm.fit(x_train, y_train)
    print("fit")
    lgb.plot_importance(
        gbm
    )
    #predict
    y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration_)
    save_submission("lgbm.csv")
    print(((y_pred - y_test) ** 2).mean() ** .5)

In [ ]:
lgbm(x_train, x_test, y_train, y_test)